<center><img src="https://drive.google.com/uc?export=view&id=1TioFdYzIbmRKAWdoXoSFld50Vne1pQ60" /></center>

<h1 style="color:#6495ED;">Python - SQLAlchemy</h1>

<p>SQLAlchemy é uma biblioteca de mapeamento objeto-relacional para Python. <br />
Neste é possível utilizar bancos de dados com Python, tanto através da definição de classes quanto através de código SQL.</p>


<p><span style="color: crimson;">Observação:</span> Nos exemplos deste notebook será utilizado o SQLite e a tabela abaixo descrita. Para visualização do banco de dados será utilizado o SQLiteStudio.</p>

<pre>
CREATE TABLE carro(
    cd_carro INTEGER PRIMARY KEY AUTOINCREMENT,
    nm_marca VARCHAR(50),
    nr_placa VARCHAR(10),
    nm_cor VARCHAR(10)
);
</pre>

<p>Antes de iniciar, é necessário instalar o SQLAlchemy:</p>

In [ ]:
%pip install sqlalchemy

<h3 style="color:#6495ED;">Criação da engine:</h3>

<p>Para conectar a um banco de dados suportado pelo SQLAlchemy é necessário instanciar um objeto da classe <span style="color:#3DCAB5;">Engine</span> utilizando a função <span style="color:khaki;">create_engine</span><br />
A string de conexão que é passada como parâmetro para o <span style="color:khaki;">create_engine</span> muda de acordo com o banco de dados alvo.</p>

In [4]:
from sqlalchemy import create_engine

DATABASE_NAME = "aula.db"

engine = create_engine(f"sqlite:///{DATABASE_NAME}")
engine

Engine(sqlite:///aula.db)

<h3 style="color:#6495ED;">Criação da sessão:</h3>

<p>Quando criada uma engine, ela pode utilizada para a criação de uma <span style="color:#3DCAB5;">Sessão</span> utilizando a função <span style="color:khaki;">sessionmaker</span>.<br />
Comando SQL devem ser executados a partir de uma sessão.</p>

In [5]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session() # Cria uma nova sessão!
session.execute
session

<h3 style="color:#6495ED;">Executando comandos SQL:</h3>

<p>Utiliza-se o método <span style="color:khaki;">execute</span> para executar comandos SQL no banco de dados.<br />
Sessões funcionam no formato transaction, de forma que é necessário <span style="color:khaki;">commit</span> para executar comandos DML no BD.</p>

In [6]:
session.execute("""CREATE TABLE carro(
    cd_carro INTEGER PRIMARY KEY AUTOINCREMENT,
    nm_marca VARCHAR(50),
    nr_placa VARCHAR(10),
    nm_cor VARCHAR(10)
    );""")

/tmp/ipykernel_17991/2914671523.py:1: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  session.execute("""CREATE TABLE carro(


In [7]:
session.execute("""INSERT INTO carro (nm_marca, nr_placa, nm_cor) VALUES ('Fiat', 'ABC-1234', 'Prata')""")
session.execute("""INSERT INTO carro (nm_marca, nr_placa, nm_cor) VALUES ('Volvo', 'ABC-9876', 'Branco')""")
session.commit()

<p>Caso haja um erro, é possível retornar a sessão ao estado anterior, não propagando as modificações, utilizando o <span style="color:khaki;">rollback</span>:</p>

In [8]:
retorno = session.execute("""INSERT INTO carro (nm_marca, nr_placa, nm_cor) VALUES ('Ford', 'ABC-4567', 'Preto')""")
session.rollback()

Consulta:

In [9]:
results = session.execute("""SELECT * FROM carro""")
results

In [10]:
for result in results:
    print(result)

(1, 'Fiat', 'ABC-1234', 'Prata')
(2, 'Volvo', 'ABC-9876', 'Branco')


In [11]:
class Carro:

    def __init__(self, cd_carro, nm_marca, nr_placa, nm_cor) -> None:
        self.cd_carro = cd_carro
        self.nm_marca = nm_marca
        self.nr_placa = nr_placa
        self.nm_cor = nm_cor

In [12]:
results = session.execute("""SELECT * FROM carro""")
results

In [13]:
for result in results:
    carro = Carro(result[0], result[1], result[2], result[3])
    print(carro.__dict__)

{'cd_carro': 1, 'nm_marca': 'Fiat', 'nr_placa': 'ABC-1234', 'nm_cor': 'Prata'}
{'cd_carro': 2, 'nm_marca': 'Volvo', 'nr_placa': 'ABC-9876', 'nm_cor': 'Branco'}


In [14]:
results = session.execute("""SELECT * FROM carro WHERE cd_carro = 1 LIMIT 1""")
results.__next__()

(1, 'Fiat', 'ABC-1234', 'Prata')

<p><b>Uma sessão deve sempre ser encerrada, utilizando o <span style="color:khaki;">close</span>:</b></p>

In [ ]:
session.close()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
engine = create_engine("""sqlite:///aula.db""")
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
session.execute("""CREATE TABLE cliente(
    cd_cliente INTEGER PRIMARY KEY AUTOINCREMENT,
    nm_cliente VARCHAR(50) NOT NULL,
    nr_cpf VARCHAR(11) NOT NULL,
    dsc_endereco VARCHAR(70),
    renda NUMBER
    )""")

In [ ]:
session.execute("""INSERT INTO cliente (nm_cliente, nr_cpf, dsc_endereco, renda)
    VALUES ('Fulano', '10', 'Rua Bla 10', 1200)""")
session.execute("""INSERT INTO cliente (nm_cliente, nr_cpf, dsc_endereco, renda)
    VALUES ('Beltrano', '11', 'Rua Ble 100', 1400)""")
session.commit()
session.close()

In [ ]:
class Cliente:
    
    def __init__(self, nm_cliente, nr_cpf, dsc_endereco, renda, cd_cliente=None) -> None:
        self.cd_cliente = cd_cliente
        self.nm_cliente = nm_cliente
        self.nr_cpf = nr_cpf
        self.dsc_endereco = dsc_endereco
        self.renda = renda

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from typing import List, Union

class ClienteDAO:

    def __init__(self) -> None:
        DATABASE_NAME = "aula.db"
        Session = sessionmaker(bind=create_engine(f"sqlite:///{DATABASE_NAME}"))
        self.session = Session()
    
    def __del__(self) -> None:
        self.session.close()

    def save(self, cliente: Cliente) -> None:
        self.session.execute(f"""INSERT INTO cliente (nm_cliente, nr_cpf, dsc_endereco, renda)
        VALUES ('{cliente.nm_cliente}', '{cliente.nr_cpf}', '{cliente.dsc_endereco}', '{cliente.renda}')""")
        self.session.commit()

    def query(self, cd_cliente: int) -> Union[Cliente,None]:
        results = self.session.execute(f"""SELECT nm_cliente, nr_cpf, dsc_endereco, renda, cd_cliente FROM cliente
                                           WHERE cd_cliente = {cd_cliente} LIMIT 1""")
        try:
            result = results.__next__()
            cliente = Cliente(result[0], result[1], result[2], result[3], result[4])
            return cliente
        except StopIteration:
            return None

    def query_all(self) -> List[Cliente]:
        resultados = []
        results = self.session.execute(f"""SELECT nm_cliente, nr_cpf, dsc_endereco, renda, cd_cliente FROM cliente""")
        for result in results:
            cliente = Cliente(result[0], result[1], result[2], result[3], result[4])
            resultados.append(cliente)
        return resultados


In [ ]:
cliente = Cliente('Cicrano', '20', 'Rua Sei lá 201', 1500)
dao = ClienteDAO()
dao.save(cliente)

In [ ]:
cliente = dao.query(1)
cliente.__dict__

In [ ]:
clientes = dao.query_all()
for cliente in clientes:
    print(cliente.__dict__)

<h3 style="color:#6495ED;">ORM:</h3>

<p>Além de poder se executar comandos SQL o SQLAlchemy também possuí um pacote de ORM ( Object-relational mapping ) que permite mapear as tabelas do banco em classes e objetos. Para isso, utilizamos a função <span style="color:khaki";>declarative_base</span> do pacote <span style="color:#3DCAB5;">sqlalchemy.orm</span></p>

In [ ]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String, Date

class CarroTable(Base):
    __tablename__ = "carro"

    cd_carro = Column("cd_carro", Integer, primary_key=True)
    nm_marca = Column("nm_marca", String(50))
    nr_placa = Column("nr_placa", String(10))
    nm_cor = Column("nm_cor", String(10))

In [ ]:
carro = CarroTable()
carro.nm_marca = "Ford"
carro.nr_placa = "ABE-2222"
carro.nm_cor = "Azul"

In [ ]:
from sqlalchemy import create_engine

DATABASE_NAME = "aula.db"

engine = create_engine(f"sqlite:///{DATABASE_NAME}")
engine

INSERT:

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
session.add(carro)
session.commit()
session.close()

SELECT:

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).all()

for result in results:
    print(result.__dict__)

session.close()

SELECT BY ID:

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).filter_by(cd_carro=1)

for result in results:
    print(result.__dict__)

session.close()

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).filter_by(nm_cor='Azul')

for result in results:
    print(result.__dict__)

session.close()

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).filter(CarroTable.nm_cor =='Azul')

for result in results:
    print(result.__dict__)

session.close()

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

results = session.query(CarroTable).filter(CarroTable.nm_cor != 'Azul')

for result in results:
    print(result.__dict__)

session.close()

<h3 style="color:crimson;">Exercícios:</h3>

<p>1) Crie uma estrutura de classes para a entidade Cliente.</p>
<span>a) A entidade Cliente possui os atributos: nome, cpf, endereço e renda</span><br />
<span>b) Utilizando SQLAlchemy, criar a estrtura de uma tabela cliente<span><br />
<span>c) Inserir novos clientes no banco de dados<span><br />
<span>d) Selecionar todos os clientes<span><br />
<span>e) Selecionar um cliente específico pelo ID<span><br />
<span>f) Selecionar clientes cuja renda seja maior que R$1200 e menor ou igual a R$1500<span><br />


In [ ]:
from sqlalchemy.orm import declarative_base

Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String, Float

class Cliente(Base):
    __tablename__ = "cliente"

    cd_cliente = Column("cd_cliente", Integer, primary_key=True)
    nm_cliente = Column("nm_cliente", String(50))
    nr_cpf = Column("nr_cpf", String(11))
    dsc_endereco = Column("dsc_endereco", String(70))
    renda = Column("renda", Float)

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
cliente = Cliente()
cliente.nm_cliente = "Batman"
cliente.nr_cpf = "100"
cliente.dsc_endereco = "BatCaverna"
cliente.renda = 1

In [ ]:
session.add(cliente)
session.commit()

In [ ]:
results = session.query(Cliente).all()
for result in results:
    print(result.__dict__)

In [ ]:
results = session.query(Cliente).filter_by(cd_cliente=4)
for result in results:
    print(result.__dict__)

In [ ]:
results = session.query(Cliente).filter(Cliente.renda > 1200, Cliente.renda <= 1500)
for result in results:
    print(result.__dict__)

In [ ]:
session.close()

<h3 style="color:#6495ED;">Dataclasses:</h3>

In [ ]:
from dataclasses import dataclass

In [ ]:
from typing import Optional

@dataclass
class Carro:
    cd_carro: int
    nm_marca: str
    nr_placa: str
    nm_cor: Optional[str] = None

In [ ]:
carro = Carro(1, "Citroen", "ABC-2020", "Vermelho")

In [ ]:
print(carro)

<h3 style="color:#6495ED;">Agrupando as duas estruturas:</h3>

In [ ]:
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [ ]:
class CarroTable(Base):
    __tablename__ = "carro"

    cd_carro = Column("cd_carro", Integer, primary_key=True)
    nm_marca = Column("nm_marca", String(50))
    nr_placa = Column("nr_placa", String(10))
    nm_cor = Column("nm_cor", String(10))

In [ ]:
@dataclass
class Carro(CarroTable):
    cd_carro: int
    nm_marca: str
    nr_placa: str
    nm_cor: Optional[str] = None

In [ ]:
carro = Carro(4, "Citroen", nr_placa="ABC-2020", nm_cor="Prata")

<h3>Links relevantes</h3><br />
SQLite:<br />
<a href="https://www.sqlite.org/index.html">Documentação do SQLite</a><br />
<a href="https://sqlitestudio.pl/">Download do SQLite Studio</a><br />

SQLAlchemy:<br />
<a href="https://docs.sqlalchemy.org/en/14/">Site do SQLAlchemy - Documentação da versão 1.4</a><br />
<a href="https://www.treinaweb.com.br/blog/o-que-e-o-sqlalchemy">O que é o SQLAlchemy?</a><br />

Dataclasses:<br />
<a href="https://pt.stackoverflow.com/questions/376306/o-que-s%C3%A3o-dataclasses-e-quando-utiliz%C3%A1-las">Stack Overflow - Explicação geral sobre dataclasses</a><br />
<a href="https://realpython.com/python-data-classes/">Real Python - Explicação de dataclasses com alternativas (inglês)</a><br />
<a href="[19:44] al-zanatta (Convidado)
https://docs.python.org/3/library/dataclasses.html">Docs Python - Documentação de Dataclasses (inglês)</a><br />

<a href="https://docs.sqlalchemy.org/en/14/orm/dataclasses.html">SQLAlchemy 1.4 + Dataclasses utilizando mapeamento imperativo</a><br />

Design Patterns:<br />
<a href="https://www.guj.com.br/t/o-que-e-dao-dto/64094/2">Padrão DAO e DTO - Forum do Java</a>